# Credit Card Fraud Detection with AutoXGB
## 1. Data Acquisition and Processing
___

### (i) Setup dependencies and dataset

In [1]:
# Library imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import pickle
import datetime
import shutil

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

#### Download raw data
- Data source: https://github.com/Fraud-Detection-Handbook/simulated-data-transformed

In [2]:
# Download raw data
dst_folder = 'data/raw'

if not os.path.exists(dst_folder):
    os.makedirs(dst_folder)
    !git clone https://github.com/Fraud-Detection-Handbook/simulated-data-transformed/ data/raw

In [3]:
os.listdir(f'{dst_folder}/data')

['2018-04-01.pkl',
 '2018-04-02.pkl',
 '2018-04-03.pkl',
 '2018-04-04.pkl',
 '2018-04-05.pkl',
 '2018-04-06.pkl',
 '2018-04-07.pkl',
 '2018-04-08.pkl',
 '2018-04-09.pkl',
 '2018-04-10.pkl',
 '2018-04-11.pkl',
 '2018-04-12.pkl',
 '2018-04-13.pkl',
 '2018-04-14.pkl',
 '2018-04-15.pkl',
 '2018-04-16.pkl',
 '2018-04-17.pkl',
 '2018-04-18.pkl',
 '2018-04-19.pkl',
 '2018-04-20.pkl',
 '2018-04-21.pkl',
 '2018-04-22.pkl',
 '2018-04-23.pkl',
 '2018-04-24.pkl',
 '2018-04-25.pkl',
 '2018-04-26.pkl',
 '2018-04-27.pkl',
 '2018-04-28.pkl',
 '2018-04-29.pkl',
 '2018-04-30.pkl',
 '2018-05-01.pkl',
 '2018-05-02.pkl',
 '2018-05-03.pkl',
 '2018-05-04.pkl',
 '2018-05-05.pkl',
 '2018-05-06.pkl',
 '2018-05-07.pkl',
 '2018-05-08.pkl',
 '2018-05-09.pkl',
 '2018-05-10.pkl',
 '2018-05-11.pkl',
 '2018-05-12.pkl',
 '2018-05-13.pkl',
 '2018-05-14.pkl',
 '2018-05-15.pkl',
 '2018-05-16.pkl',
 '2018-05-17.pkl',
 '2018-05-18.pkl',
 '2018-05-19.pkl',
 '2018-05-20.pkl',
 '2018-05-21.pkl',
 '2018-05-22.pkl',
 '2018-05-23

#### Create train and test datasets
- ~4 month period for **train** set: 2018-05-01 to 2018-09-10
- 1 week Feedback Delay between train and test set: 2018-09-11 to 2017-09-17
- 1 week period for **test** set: 2018-09-18 to 2018-09-24

*Details on Feedback Delay Period*
- We choose our test set to take place one week after the last transaction of the training set. In a fraud detection context, this period separating the training and test set is referred to as the delay period or feedback delay. 
- It accounts for the fact that, in a real-world fraud detection system, the label of a transaction (fraudulent or genuine) is only known after a customer complaint, or thanks to the result of a fraud investigation. 
- Therefore, in a realistic scenario, the annotated data available to train a model and start making prediction for a given day are anterior to that day minus the delay period. 
- A one-week delay is, to a first approximation, a reasonable basis. From experience, statistics generally show that most of the feedback becomes available after a one week delay and empirically, it is also the point where the negative effect of considering that zero feedback is available before and the positive effect of considering that all the feedback is available after cancel each other out.

In [4]:
# Define date range
train_date_range = pd.date_range('2018-05-01', '2018-09-10', freq='d').strftime('%Y-%m-%d').tolist()
test_date_range = pd.date_range('2018-09-18', '2018-09-24', freq='d').strftime('%Y-%m-%d').tolist()

In [5]:
# Generate train dataframe
train_df = pd.DataFrame()
for date in train_date_range:
    df = pd.read_pickle(f'{dst_folder}/data/{date}.pkl')
    train_df = train_df.append(df)

len(train_df)

1274331

In [6]:
# Generate test dataframe
test_df = pd.DataFrame()
for date in test_date_range:
    df = pd.read_pickle(f'{dst_folder}/data/{date}.pkl')
    test_df = test_df.append(df)

len(test_df)

67416

___
### (iii) Data-Preprocessing
- Baseline feature transformation already done on the simulated raw dataset based on: https://fraud-detection-handbook.github.io/fraud-detection-handbook/Chapter_3_GettingStarted/BaselineFeatureTransformation.html

In [7]:
train_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
288062,288062,2018-05-01 00:01:21,3546,2944,18.71,2592081,30,0,0,0,1,3.0,38.070000,17.0,44.430588,89.0,42.628652,0.0,0.0,12.0,0.0,36.0,0.0
288063,288063,2018-05-01 00:01:48,206,3521,18.60,2592108,30,0,0,0,1,6.0,29.988333,26.0,27.461538,104.0,26.104038,6.0,0.0,11.0,0.0,38.0,0.0
288064,288064,2018-05-01 00:02:22,2610,4470,66.67,2592142,30,0,0,0,1,6.0,46.033333,26.0,46.767692,110.0,50.019727,1.0,0.0,5.0,0.0,22.0,0.0
288065,288065,2018-05-01 00:03:15,4578,1520,79.41,2592195,30,0,0,0,1,1.0,79.410000,17.0,83.097647,69.0,91.713188,0.0,0.0,8.0,0.0,33.0,0.0
288066,288066,2018-05-01 00:03:51,1246,7809,52.08,2592231,30,0,0,0,1,9.0,54.158889,35.0,52.931714,130.0,55.928385,0.0,0.0,6.0,0.0,19.0,0.0


In [8]:
target = 'TX_FRAUD'

predictors = ['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 
              'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
              'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
              'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
              'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
              'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
              'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
              'TERMINAL_ID_RISK_30DAY_WINDOW']

In [9]:
# Keep only predictor and target columns
train_df = train_df[predictors + [target]]
test_df = test_df[predictors + [target]]

In [10]:
# Assess degree of imbalance in target label
train_df[target].value_counts()

0    1263048
1      11283
Name: TX_FRAUD, dtype: int64

Significant degree of target label imbalance (Fraud cases only 0.9% of entire dataset)

#### Perform sampling strategy
1. SMOTE oversampling to make minority labels 10% of total dataset, then
2. Random undersampling to make minority 50% of majority class

Final sampled ratio will be majority:minority = 2:1

In [11]:
# Define sampling techniques
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)

steps = [('over', over), ('under', under)]
pipeline = Pipeline(steps=steps)

In [12]:
# Rearrange dataset into X, y
X_train = train_df[predictors]
y_train = train_df[target]

In [13]:
# Execute sampling strategy
X_train, y_train = pipeline.fit_resample(X_train, y_train)

In [14]:
len(X_train)

378912

In [15]:
y_train.value_counts()

0    252608
1    126304
Name: TX_FRAUD, dtype: int64

### Save Processed Datasets

In [16]:
processed_folder = 'data/processed'
if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)
    
test_df.to_csv(f'{processed_folder}/test.csv', index=False)

In [17]:
# Combine X_train and y_train together for saving as processed train dataset
y_train_df = pd.DataFrame(y_train, columns=['TX_FRAUD'])
train_processed = pd.concat([X_train, y_train_df], axis=1)
train_processed.to_csv(f'{processed_folder}/train.csv', index=False)

#### References
- https://www.kaggle.com/mlg-ulb/creditcardfraud
- https://fraud-detection-handbook.github.io/fraud-detection-handbook/Chapter_References/shared_functions.html
- https://www.kaggle.com/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets
- https://github.com/Fraud-Detection-Handbook/simulated-data-raw/tree/main/data